In [333]:
import redis 
import json

In [334]:
r = redis.Redis(
    host='redis',
    port=6379,
    db=0,
    decode_responses=True)
r.ping()

True

In [335]:
def add_player(id,name,country,games_played, score):
    player_data = {     
        "name" : name,
        "country" : country,
        "games_played" : 0,
        "score" : score,
    }
    
    player_key = f"player:{id}"
        
    r.hset(player_key,mapping=player_data)

    leaderboard_key = "leaderboard"
    r.zadd(leaderboard_key,{id : score})

In [336]:
add_player(1,"jorge","españa",13,1000)
add_player(2,"Pepe","españa",13,1000)

In [337]:
def update_score(id,points):
    player_key = f"player:{id}"
    player_data = r.hgetall(player_key)
    player_data["score"] = int(player_data["score"]) + points
    player_data["games_played"] = int(player_data["games_played"]) + 1
    r.hset(player_key,mapping=player_data)

    r.zadd("leaderboard",{id:int(player_data["score"])})

In [338]:
update_score("2",39000)

In [339]:
def player_info(id):
    player_key = f"player:{id}"

    info = r.hgetall(player_key)


    for key, value in info.items():
        print(key +" > "+value)
       

In [340]:
player_info(1)

name > jorge
country > españa
games_played > 0
score > 1000


In [341]:
def show_top_players(n):
    top_players = r.zrevrange("leaderboard", 0, n -1, withscores=True)

    if not top_players:
        print("El ranking está vacio")
        return
    
    for i ,(id,score) in enumerate(top_players):
        player_key = f"player:{id}"

        res = r.hgetall(player_key)

        print(f"#{i+1} {res['name']} ({res['country']})")
        print(f"ID: {id} Score: {score}")

In [342]:
show_top_players(2)

#1 Pepe (españa)
ID: 2 Score: 40000.0
#2 jorge (españa)
ID: 1 Score: 1000.0


In [343]:
import datetime as date

def get_date_key(date_obj):
    return f"unique:players:{date_obj.isoformat()}"

def register_login(id):

    today_date = date.date.today()
    
    today_key = get_date_key(today_date)

    changed = r.pfadd(today_key,id)

In [344]:
register_login(2)

In [353]:
def count_unique_logins(date_str):
    key = f"unique:players:{date_str}"
    count = r.pfcount(key)

    print(f"Jugadores unicos el {date_str:{count}}")

In [357]:
count_unique_logins("2025-11-11")

Jugadores unicos el 2025-11-11


In [358]:
def weekly_report(dates_list):
    keys_to_merge = [f"unique:players:{d}" for d in dates_list]

    report_key = "unique:players:week_report"
    r.pfmerge(report_key, *keys_to_merge)

    total_unique = r.pfcount(report_key)

    print(f"Reporte semanal (Dias: {len(dates_list)})")
    print(f"Total de jugadores únicos (aprox): {total_unique}")

    r.delete(report_key)
    return total_unique

In [359]:
weekly_report(["2025-11-10", "2025-11-11"])

Reporte semanal (Dias: 2)
Total de jugadores únicos (aprox): 2


2